## blazegraph

requires some setup & maintaince  
blazegraph.deb fetched from  ... i.e.   

https://downloads.sourceforge.net/project/bigdata/bigdata/2.1.4/blazegraph.deb?r=&ts=1500416967&use_mirror=managedway

then on ubuntu derived machines 

    sudo apt-get install ./blazegraph_2.1.1.deb  

or debian machines 

    sudo dpkg -i blazegraph_2.1.1.deb



If you have installed blazegraph,  
and have loaded the jaspar.nt RDF file  

To start/restart blazegraph if it is not persistant.

In [1]:
# sudo service blazegraph start

To remove an existing store from the default namespace 'kb'

In [3]:
# curl -X DELETE http://localhost:9999/blazegraph/namespace/kb

to load into the default namespace one easy way is starting at:  

http://localhost:9999/blazegraph/#update  

pasteing the full path to the RDF file into the browser text area (not url field). 

Choose ```Type: File Path or URL``` below the ```Browse``` button  

Note1: browsing to the file with the button on that page will not work unless the dataset is tiny.

    /home/tomc/Projects/OHSU/TransmedTranslator/FA_gene_set/jaspar.nt

Note2: the RDF file and path need to be readable by the blazegraph user so if you do not want to open up where the file is generated move it then give lax permisions elsewhere.  

    cp jaspar.nt /tmp/jaspar.nt
    chmod a+r /tmp/jaspar.nt


Once the data is loaded you can confirm some basic stats with:

In [1]:
curl http://localhost:9999/blazegraph/namespace/kb/sparql

<?xml version="1.0" encoding="UTF-8"?>
<rdf:RDF
	xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#">

<rdf:Description rdf:nodeID="service">
	<rdf:type rdf:resource="http://www.w3.org/ns/sparql-service-description#Service"/>
	<endpoint xmlns="http://www.w3.org/ns/sparql-service-description#" rdf:resource="http://localhost:9999/blazegraph/namespace/kb/sparql"/>
	<endpoint xmlns="http://www.w3.org/ns/sparql-service-description#" rdf:resource="http://localhost:9999/bigdata/LBS/namespace/kb/sparql"/>
	<supportedLanguage xmlns="http://www.w3.org/ns/sparql-service-description#" rdf:resource="http://www.w3.org/ns/sparql-service-description#SPARQL10Query"/>
	<supportedLanguage xmlns="http://www.w3.org/ns/sparql-service-description#" rdf:resource="http://www.w3.org/ns/sparql-service-description#SPARQL11Query"/>
	<supportedLanguage xmlns="http://www.w3.org/ns/sparql-service-description#" rdf:resource="http://www.w3.org/ns/sparql-service-description#SPARQL11Update"/>
	<feature xmlns="http://

</rdf:Description>

<rdf:Description rdf:nodeID="node1blbakd64x6">
	<property xmlns="http://rdfs.org/ns/void#" rdf:resource="http://purl.obolibrary.org/obo/SO_adjacent_to"/>
	<triples xmlns="http://rdfs.org/ns/void#" rdf:datatype="http://www.w3.org/2001/XMLSchema#long">46477</triples>
</rdf:Description>

<rdf:Description rdf:nodeID="defaultGraph">
	<propertyPartition xmlns="http://rdfs.org/ns/void#" rdf:nodeID="node1blbakd64x7"/>
</rdf:Description>

<rdf:Description rdf:nodeID="node1blbakd64x7">
	<property xmlns="http://rdfs.org/ns/void#" rdf:resource="http://www.geneontology.org/formats/oboInOwl#hasDbXref"/>
	<triples xmlns="http://rdfs.org/ns/void#" rdf:datatype="http://www.w3.org/2001/XMLSchema#long">34800</triples>
</rdf:Description>

<rdf:Description rdf:nodeID="defaultGraph">
	<propertyPartition xmlns="http://rdfs.org/ns/void#" rdf:nodeID="node1blbakd64x8"/>
</rdf:Description>

<rdf:Description rdf:nodeID="node1blbakd64x8">
	<property xmlns="http://rdfs.org/ns/void#" rdf:resource

A maintaince strategy could be as simple as a makefile  
with a dependency beteeen the input file and this output.  
which requires moving away from loading via the web interface.



In [13]:
# taken from:
# https://wiki.blazegraph.com/wiki/index.php/Bulk_Data_Load#Command_line

curl -X DELETE http://localhost:9999/blazegraph/namespace/kb
echo ""
/usr/bin/loadRestAPI.sh /tmp/jaspar.nt

# and it is broken ...
# in /usr/bin/loadRestAPI.sh
# repeats `RWStore.properties` file name twice
# propertyFile=/etc/blazegraph/RWStore.properties/RWStore.properties

# looks like they know / are addressing
# https://github.com/blazegraph/database/issues/54
# ...
# which brings us to their next bug
# java.util.concurrent.ExecutionException: 
# java.lang.RuntimeException: 
# java.util.concurrent.ExecutionException: 
# java.lang.RuntimeException: 
# java.util.concurrent.ExecutionException: 
# java.lang.IllegalArgumentException: 
# context not bound: < TermId(118U), TermId(113U), TermId(69029U) 
# : Explicit >
# ...
# and so on for hundreds of lines without a clear hint of what the problem is
# 
# At the very bottom of the page 
# https://wiki.blazegraph.com/wiki/index.php/Bulk_Data_Load#Command_line
# mentions "context not bound Exception"
# points to: 
# https://wiki.blazegraph.com/wiki/index.php/REST_API#Context_Not_Bound_Error_.28Quads_mode_without_defaultGraph.29
#
# If you receive an error such as below, 
# it means that you are loading a quads mode namespace 
# without specifying the defaultGraph.
#
# ... because one can't count to three in java ...
#
# I'm putting 
#
#     defaultGraph=http://example.org
#
# in /usr/bin/loadRestAPI.sh since I have already been changing it 

curl http://localhost:9999/blazegraph/namespace/kb/sparql > BlazeGraphServiceDescription.xml

DELETED: kb
Loading with properties...
quiet=false
verbose=0
closure=false
durableQueues=true
#Needed for quads
defaultGraph=http://example.org
com.bigdata.rdf.store.DataLoader.flush=false
com.bigdata.rdf.store.DataLoader.bufferCapacity=100000
com.bigdata.rdf.store.DataLoader.queueCapacity=10
#Namespace to load
namespace=kb
#Files to load
fileOrDirs=/tmp/jaspar.nt
#Property file (if creating a new namespace)
propertyFile=/etc/blazegraph/RWStore.properties
CREATED: kb<?xml version="1.0"?><data modified="0" milliseconds="15345"/>DATALOADER-SERVLET: Loaded kb with properties: /etc/blazegraph/RWStore.properties  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 11183    0 11183    0     0   253k      0 --:--:-- --:--:-- --:--:--  260k


And it is back

In [2]:
head BlazeGraphServiceDescription.xml

<?xml version="1.0" encoding="UTF-8"?>
<rdf:RDF
	xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#">

<rdf:Description rdf:nodeID="service">
	<rdf:type rdf:resource="http://www.w3.org/ns/sparql-service-description#Service"/>
	<endpoint xmlns="http://www.w3.org/ns/sparql-service-description#" rdf:resource="http://localhost:9999/blazegraph/namespace/kb/sparql"/>
	<endpoint xmlns="http://www.w3.org/ns/sparql-service-description#" rdf:resource="http://localhost:9999/bigdata/LBS/namespace/kb/sparql"/>
	<supportedLanguage xmlns="http://www.w3.org/ns/sparql-service-description#" rdf:resource="http://www.w3.org/ns/sparql-service-description#SPARQL10Query"/>
	<supportedLanguage xmlns="http://www.w3.org/ns/sparql-service-description#" rdf:resource="http://www.w3.org/ns/sparql-service-description#SPARQL11Query"/>


Some queries to run in the BG Workbench interface at 
http://localhost:9999/blazegraph/#query
include


```
PREFIX SO: <http://purl.obolibrary.org/obo/SO_>
PREFIX RO: <http://purl.obolibrary.org/obo/RO_>
PREFIX SWO: <http://www.ebi.ac.uk/efo/swo/SWO_>
PREFIX NCBIGene: <http://www.ncbi.nlm.nih.gov/gene/>

SELECT (COUNT(?gene) as ?count)
  WHERE {
  	?gene SO:adjacent_to ?region1 .
    ?region1 RO:0002350 ?pairwise .
    ?region2 RO:0002350 ?pairwise .
    FILTER(?region1 != ?region2)
    ?gene2 SO:adjacent_to ?region2 .
    FILTER(?gene != ?gene2)
	?pairwise SWO:0000425 '1' .
  }

count
19790
```
SPARQL path notation works

```    
    select ?g1 ?g2
	    WHERE {
        ?g1 SO:adjacent_to/RO:0002350/^RO:0002350/^SO:adjacent_to ?g2 .
        FILTER(?g1 != ?g2)
      }
    LIMIT 3
```
````
g1	                                        g2
<http://www.ncbi.nlm.nih.gov/gene/10200>	<http://www.ncbi.nlm.nih.gov/gene/3646>
<http://www.ncbi.nlm.nih.gov/gene/10200>	<http://www.ncbi.nlm.nih.gov/gene/23077>
<http://www.ncbi.nlm.nih.gov/gene/10200>	<http://www.ncbi.nlm.nih.gov/gene/55269>
```

```
Look for genes related to FANCA

select ?g2
	WHERE {
  	NCBIGene:2175 SO:adjacent_to/RO:0002350/^RO:0002350/^SO:adjacent_to ?g2 .
    FILTER(NCBIGene:2175 != ?g2)
  }
```

```
g2
<http://www.ncbi.nlm.nih.gov/gene/8795>
<http://www.ncbi.nlm.nih.gov/gene/348995>
<http://www.ncbi.nlm.nih.gov/gene/5166>

...

```